# Create AMP Zone Mapping

This notebook creates an enhanced lot mapping file that includes AMP zone name aliases.

## Purpose

AMP uses detailed zone names like "Green 1 PACCAR South" while lot_mapping uses simple names like "Green 1". This notebook creates a mapping table that allows any dataset to find the correct lot number regardless of how the zone is named.

## Output

- `amp_zone_aliases.csv`: Mapping of AMP zone names to lot numbers and standardized zone names
- Can be used by any notebook to map AMP, LPR, or ticket data to consistent lot numbers

In [1]:
import pandas as pd
import numpy as np

## Load Base Lot Mapping

In [2]:
# Load the base lot mapping
lot_mapping = pd.read_csv('../data/lot_mapping_enhanced.csv')

print(f"Base lot mapping: {len(lot_mapping)} lots")
print(f"\nColumns: {list(lot_mapping.columns)}")
print(f"\nSample:")
print(lot_mapping.head())

Base lot mapping: 188 lots

Columns: ['Lot_number', 'Zone_Name', 'zone_type', 'capacity', 'location_description', 'is_dorm_parking', 'alternative_location_description', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', '21,50,55,56,101,178,179']

Sample:
   Lot_number Zone_Name zone_type  capacity location_description  \
0           1    Grey 2    Permit       NaN         ROGERS-ORTON   
1           2   Green 2    Permit       NaN           MCALLISTER   
2           3   Green 2    Permit       NaN        KRUEGEL SOUTH   
3           4   Green 2    Permit       NaN        KRUEGEL NORTH   
4           5     Red 2    Permit       NaN   GANNON/GOLDSWORTHY   

   is_dorm_parking     alternative_location_description  Unnamed: 7  \
0                1                                  NaN         NaN   
1                0  Green 2 KMac Lot|Green 2 McAllister         NaN   
2                0                                  NaN         NaN   
3              

## Convert to DataFrame and Save

In [8]:
# Convert dictionary to DataFrame
alias_data = []
for amp_zone, (lot_num, std_zone) in amp_zone_aliases.items():
    alias_data.append({
        'AMP_Zone_Name': amp_zone,
        'Lot_number': int(lot_num),  # Convert to int to match lot_mapping
        'Standard_Zone_Name': std_zone
    })

amp_aliases_df = pd.DataFrame(alias_data)

# Merge with lot_mapping to get additional metadata
amp_aliases_df = amp_aliases_df.merge(
    lot_mapping[['Lot_number', 'zone_type', 'is_dorm_parking']],
    on='Lot_number',
    how='left'
)

print(f"\nAMP Zone Aliases DataFrame:")
print(amp_aliases_df.head(10))
print(f"\nShape: {amp_aliases_df.shape}")
print(f"\nZone types: {amp_aliases_df['zone_type'].value_counts()}")


AMP Zone Aliases DataFrame:
                                AMP_Zone_Name  Lot_number Standard_Zone_Name  \
0                        Green 1 PACCAR South          10            Green 1   
1                          Green 1 Bustad Lot          10            Green 1   
2          Green 1 Rotunda/Gannon-Goldsworthy          10            Green 1   
3                            Green 2 KMac Lot          36            Green 2   
4                          Green 2 McAllister          36            Green 2   
5       Green 3: (Gravel lot) College/Spokane           9            Green 3   
6      Green 3: (Gravel lot) Columbia/Spokane           9            Green 3   
7       Green 3: College Avenue -or- Idaho St           9            Green 3   
8  Green 3: Washington St street-side & Spark           9            Green 3   
9           Green 3: Thermal Fluids/Dana West           9            Green 3   

  zone_type  is_dorm_parking  
0    Permit                0  
1    Permit                0

In [9]:
# Save to CSV
output_file = '../data/amp_zone_aliases.csv'
amp_aliases_df.to_csv(output_file, index=False)

print(f"Saved AMP zone aliases to: {output_file}")
print(f"Total aliases: {len(amp_aliases_df)}")

Saved AMP zone aliases to: ../data/amp_zone_aliases.csv
Total aliases: 59
